In [1]:
#attempt to modify NTRU Encrypt by using R_p = F_p[S_N] rather than R_p = F_p[C_N] = F_p[x]/(x^N-1)

In [2]:
#the group of units of the symmetric group algebra are computed in Corollary 3.1, 3.2 of
#"Structure of the Unit Group of the Symmetric Group Algebra", Abhilash, Nandakumar

In [3]:
homomorphic_encryption = False

In [4]:
#define the parameters
N=7; p=3; q=41; d=2
assert N.is_prime()
assert p.is_prime()
assert gcd(N,q) == 1 
assert gcd(p,q) == 1
assert q > (6*d+1)*p

In [5]:
#define the group
G = CyclicPermutationGroup(N)

In [6]:
#create the polynomial ring \Z[x]/(x^N-1)
R = G.algebra(ZZ); R

Algebra of Cyclic group of order 7 as a permutation group over Integer Ring

In [7]:
#create the quotient polynomial ring F_p[S_N]
R_p = G.algebra(GF(p)); R_p

Algebra of Cyclic group of order 7 as a permutation group over Finite Field of size 3

In [8]:
#create the symmetric group algebra F_q[S_N]
R_q = G.algebra(Zmod(q)); R_q

Algebra of Cyclic group of order 7 as a permutation group over Ring of integers modulo 41

In [9]:
#a ternary polynomial in R = \Z[x]/(x^N-1) has d1 coeffs = +1, d2 coeffs = -1
def is_ternary(a,d1,d2):
    assert a in R
    coeffs = [pair[1] for pair in list(a)]
    return list(coeffs).count(+1) == d1 and list(coeffs).count(-1) == d2

In [10]:
#create a random unit of the symmetric group algebra over a finite field
#essentially if K_q[S_N] \cong M_{n_1}(K_{q^{m_1}}) \oplus ... \oplus M_{n_r}(K_{q^{m_r}}) then
#\mathcal{U}(K_q[S_N]) \cong GL_{n_1}(K_{q^{m_1}}) \oplus ... \oplus GL_{n_r}(K_{q^{m_r}})
#and the number n_k, m_k can be read off from the Brattelli diagram.
#however, only ~1/p elements are not invertible by determinant considerations, so most elements should be invertible
def random_unit_group_algebra(p,n,max_iter=1,ternary=False,d1=0,d2=0):
    assert p.is_prime()
    for i in range(max_iter):
        if ternary:
            random_element = rand_ternary(N,d1,d2)
        else:
            random_element = R.random_element()
        try:
            R_p(random_element).inverse()
            return random_element
        except ValueError:
            continue
    print("Could not find invertible element of symmetric group algebra.")

In [11]:
#choose a random element of \mathcal{T}(d_1,d_2)
def rand_ternary(N,d1,d2):
    assert N >= d1 + d2
    G = R.group()
    group_order = G.cardinality()
    coeffs = (group_order-(d1+d2))*[0] + d1*[+1] + d2*[-1]
    sigma = Permutations(group_order).random_element()
    r = sum([coeffs[sigma[i]-1]*R(G[i]) for i in range(group_order)])
    assert is_ternary(r,d1,d2)
    return r

In [12]:
#create an element f of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
f = random_unit_group_algebra(p,N,max_iter=100,ternary=True,d1=d+1,d2=d)
print(f)

() + (1,2,3,4,5,6,7) - (1,3,5,7,2,4,6) + (1,4,7,3,6,2,5) - (1,5,2,6,3,7,4)


In [13]:
#create an element g of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
g = rand_ternary(N,d,d)
print(g)

(1,2,3,4,5,6,7) + (1,3,5,7,2,4,6) - (1,5,2,6,3,7,4) - (1,7,6,5,4,3,2)


In [14]:
#compute F_p(x) = f(x)^{-1} (mod p)
F_p = R_p(f).inverse(); print(F_p)
assert F_p * R_p(f) == 1

2*() + 2*(1,2,3,4,5,6,7) + (1,3,5,7,2,4,6) + 2*(1,4,7,3,6,2,5) + 2*(1,5,2,6,3,7,4) + (1,6,4,2,7,5,3)


In [15]:
#compute F_q(x) = f(x)^{-1} (mod q)
F_q = R_q(f).inverse(); print(F_q)
assert F_q * R_q(f) == 1

32*() + 23*(1,2,3,4,5,6,7) + 25*(1,3,5,7,2,4,6) + 4*(1,4,7,3,6,2,5) + 30*(1,5,2,6,3,7,4) + 13*(1,6,4,2,7,5,3) + 38*(1,7,6,5,4,3,2)


In [16]:
#define the private key used to decrypt messages
def private_key(f):
    return (f,R_q(f).inverse())

In [17]:
#define the public key using f(x) (mod p), g(x) (mod p)
def public_key(f,g,q):
    F_q = R_q(f).inverse()
    return F_q * R_q(g)

In [18]:
print(public_key(f,g,q))

24*() + 15*(1,2,3,4,5,6,7) + 38*(1,3,5,7,2,4,6) + 21*(1,4,7,3,6,2,5) + 25*(1,5,2,6,3,7,4) + 14*(1,6,4,2,7,5,3) + 27*(1,7,6,5,4,3,2)


In [48]:
#define a plaintext message m(x) with coefficients satisfying -p/2 < m_i <= p/2
#i.e. m is the center lift of a polynomial in R_p
def plaintext(m,p,N):
    assert all([-p/2 < m[i] <= +p/2 for i in range(G.cardinality())])
    return sum(m[i]*R(G[i]) for i in range(G.cardinality()))

In [60]:
#compute the ciphertext e(x)
#note m, f, g are group algebra elements
def ciphertext(m,f,g,r,p,q,N,d):
    h = public_key(f,g,q)
    e = R_q(p*h*r + m)
    return e

In [21]:
#begin decryption process
def a(f,e):
    return R_q(f*e)

In [22]:
#define the center lift of a from R_q to an element of R = \Z[x]/(x^N-1)
#map coefficients from F_q to Z/qZ (the additive group) and center lift them to \Z
def center_lift(a,modulus):
    G = R.group()
    coeffs = [a[g] for g in G]
    center_lift_coeffs = [Zmod(modulus)(coeff).lift_centered() for coeff in coeffs]
    return sum(center_lift_coeffs[i]*R(G[i]) for i in range(G.cardinality()))

In [23]:
#compute b(x) = F_p(x) * a(x) (mod p) by first center lifting a(x) to R
def b(f,a,q):
    F_p = R_p(f).inverse()
    return R_p(F_p * center_lift(a,q))

In [24]:
#define the final center_lift for decryption where e = ciphertext, f = secret key, p = prime, q = modulus
def decrypt(e,f,p,q):
    return center_lift(b(f,a(f,e),q),p)

In [25]:
#EXAMPLE

In [26]:
f_coeffs = [-1,0,1,1,-1,0,1]
f = sum([f_coeffs[i]*R(G[i]) for i in range(G.cardinality())]); f

-() + (1,3,5,7,2,4,6) + (1,4,7,3,6,2,5) - (1,5,2,6,3,7,4) + (1,7,6,5,4,3,2)

In [27]:
g_coeffs = [0,-1,-1,0,1,0,1]
g = sum([g_coeffs[i]*R(G[i]) for i in range(G.cardinality())]); g

-(1,2,3,4,5,6,7) - (1,3,5,7,2,4,6) + (1,5,2,6,3,7,4) + (1,7,6,5,4,3,2)

In [28]:
message = [1,-1,1,1,0,-1,0]; message

[1, -1, 1, 1, 0, -1, 0]

In [53]:
m = plaintext(message,p,N); m

() - (1,2,3,4,5,6,7) + (1,3,5,7,2,4,6) + (1,4,7,3,6,2,5) - (1,6,4,2,7,5,3)

In [30]:
r_coeffs = [-1,1,0,0,0,-1,1]
r = sum([r_coeffs[i]*R(G[i]) for i in range(G.cardinality())]); r

-() + (1,2,3,4,5,6,7) - (1,6,4,2,7,5,3) + (1,7,6,5,4,3,2)

In [31]:
h = public_key(f,g,q); h

30*() + 26*(1,2,3,4,5,6,7) + 8*(1,3,5,7,2,4,6) + 38*(1,4,7,3,6,2,5) + 2*(1,5,2,6,3,7,4) + 40*(1,6,4,2,7,5,3) + 20*(1,7,6,5,4,3,2)

In [32]:
R_q(p*h*r + m)

25*() + 3*(1,2,3,4,5,6,7) + 40*(1,3,5,7,2,4,6) + 2*(1,4,7,3,6,2,5) + 4*(1,5,2,6,3,7,4) + 19*(1,6,4,2,7,5,3) + 31*(1,7,6,5,4,3,2)

In [59]:
e = ciphertext(m,f,g,r,p,q,N,d); e

30*() + 26*(1,2,3,4,5,6,7) + 8*(1,3,5,7,2,4,6) + 38*(1,4,7,3,6,2,5) + 2*(1,5,2,6,3,7,4) + 40*(1,6,4,2,7,5,3) + 20*(1,7,6,5,4,3,2)
-() + (1,2,3,4,5,6,7) - (1,6,4,2,7,5,3) + (1,7,6,5,4,3,2)
() - (1,2,3,4,5,6,7) + (1,3,5,7,2,4,6) + (1,4,7,3,6,2,5) - (1,6,4,2,7,5,3)


25*() + 3*(1,2,3,4,5,6,7) + 40*(1,3,5,7,2,4,6) + 2*(1,4,7,3,6,2,5) + 4*(1,5,2,6,3,7,4) + 19*(1,6,4,2,7,5,3) + 31*(1,7,6,5,4,3,2)

In [34]:
center_lift(a(f,e),q)

3*() - 3*(1,3,5,7,2,4,6) - 6*(1,5,2,6,3,7,4) + 6*(1,6,4,2,7,5,3)

In [35]:
F_p

2*() + 2*(1,2,3,4,5,6,7) + (1,3,5,7,2,4,6) + 2*(1,4,7,3,6,2,5) + 2*(1,5,2,6,3,7,4) + (1,6,4,2,7,5,3)

In [36]:
#BUG: producing zero divisors instead of decrypted message
F_p * center_lift(a(f,e),q)

0

In [37]:
b(f,a(f,e),q)

0

In [38]:
decrypt(e,f,p,q)

0